In [1]:
!pip install langchain sentence-transformers faiss-cpu google-generativeai tqdm pandas

In [3]:
!pip install python-docx

In [5]:
from docx import Document
import re

file_path = r"\FAQ.docx"   # adjust according to your FAQ directory
doc = Document(file_path)

# 1) Flatten all paragraphs into a single text blob
raw = "\n".join(p.text for p in doc.paragraphs)

# 2) Normalise common quirks
text = raw.replace("\r", "\n")
text = re.sub(r"[ \t]+", " ", text)          # collapse spaces
text = re.sub(r"\n{2,}", "\n\n", text)       # collapse blank lines
# fix label variants like "Question :", "Answer –", etc.
replacements = {
    "Question :": "Question:",
    "Answer :": "Answer:",
    "Answer –": "Answer:",
    "Answer -": "Answer:",
    "Question –": "Question:",
    "Question -": "Question:",
}
for k, v in replacements.items():
    text = text.replace(k, v)

# 3) Split by Question: (case-insensitive, at line start)
blocks = re.split(r"(?im)^\s*question\s*:\s*", text)

faq_data = []
for block in blocks:
    block = block.strip()
    if not block:
        continue

    # find the first "Answer:" line inside this block
    m = re.search(r"(?im)^\s*answer\s*:\s*", block)
    if m:
        q = block[:m.start()].strip()
        a = block[m.end():].strip()
    else:
        # fallback: first non-empty line = question, rest = answer
        lines = [l.strip() for l in block.splitlines() if l.strip()]
        if not lines:
            continue
        q = lines[0]
        a = " ".join(lines[1:]) if len(lines) > 1 else ""

    q = re.sub(r"\s+", " ", q)
    a = re.sub(r"\s+", " ", a)

    if q and a:
        faq_data.append({"question": q, "answer": a})

print("Total Q&A pairs:", len(faq_data))
for i, qa in enumerate(faq_data, 1):
    print(f"{i}. {qa['question'][:100]}")


Total Q&A pairs: 6
1. Kenapa saya tidak boleh menonton selepas membuat bayaran?
2. Saya telah melanggan tetapi kenapa masih mempunyai iklan?
3. Bagaimana saya nak membatalkan langganan bulanan TontonUp saya?
4. Bagaimana saya nak menukar kata laluan?
5. Boleh saya melanggan apabila saya di luar negara Malaysia?
6. Mengapa ralat muncul di skrin semasa saya menikmati Tonton?


In [7]:
!pip install sentence-transformers faiss-cpu

In [9]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Load the multilingual embedding model
embedding_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

In [11]:
# Embed Q + A (not just answers)
texts = [f"Question: {it['question']}\nAnswer: {it['answer']}" for it in faq_data]

embeddings = embedding_model.encode(texts, show_progress_bar=True, convert_to_numpy=True).astype("float32")

embedding_dim = embeddings.shape[1]
index = faiss.IndexFlatL2(embedding_dim)
index.add(embeddings)

assert index.ntotal == len(faq_data), f"Index {index.ntotal} != data {len(faq_data)}"

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [13]:
import os, google.generativeai as genai
import re

# Replace with your own key
genai.configure(api_key="your_api_key_here")
#genai.configure(api_key=os.environ["GEMINI_API_KEY"])


# Load Gemini model
model = genai.GenerativeModel("models/gemini-1.5-flash")

#dont forget to replace the ""your_api_key_here" with your API_KEY

In [15]:
def ask_chatbot(user_question, top_k=3):
    # 1. Embed the user query
    query_embedding = embedding_model.encode([user_question])

    # 2. Search in FAISS
    D, I = index.search(np.array(query_embedding), top_k)

    # 3. Retrieve matching answers
    #retrieved_chunks = [faq_data[i]["answer"] for i in I[0]]
    #retrieved_context = "\n\n".join(retrieved_chunks)

   # Simple keyword overlap filter (more general)
    question_keywords = set(re.findall(r'\w+', user_question.lower()))
    retrieved_chunks = []

    for i in I[0]:
        chunk = faq_data[i]["answer"]
        chunk_words = set(re.findall(r'\w+', chunk.lower()))
        overlap = question_keywords.intersection(chunk_words)

        if len(overlap) >= 2:  # At least 2 overlapping words
            retrieved_chunks.append(chunk)

    if not retrieved_chunks:
        return "Maaf, tiada maklumat berkaitan ditemui.", ""

    retrieved_context = "\n\n".join(retrieved_chunks)

    
    # ✅ Print retrieved chunk(s) for debugging
    print("🧾 Retrieved content:\n")
    for i, chunk in enumerate(retrieved_chunks):
        print(f"Chunk {i+1}:\n{chunk}\n")

    # 4. Build prompt (updated prompt with stricter instructions)
    prompt = (
    "Sila gunakan maklumat di bawah sahaja untuk menjawab soalan pengguna. "
    "Jangan cipta jawapan yang tidak wujud dalam maklumat tersebut. "
    "Petik secara langsung maklumat penting jika perlu. "
    "Pastikan jawapan anda mengandungi semua butiran penting seperti nombor langkah atau bilangan episod. " 
    "Jika jawapan tiada dalam maklumat, balas 'Maklumat tidak tersedia'. "  
    "Jawab dalam Bahasa Melayu.\n\n"
    f"{retrieved_context}\n\n"
    f"Soalan: {user_question}\n"
    f"Jawapan:"
    )

    # 5. Generate response using Gemini
    response = model.generate_content(prompt)
    return response.text

In [17]:
response = ask_chatbot("Kenapa saya tidak boleh menonton selepas membuat bayaran?")
print(response)

🧾 Retrieved content:

Chunk 1:
Jika anda ingin mengaktifkan semula langganan untuk kandungan eksklusif, anda boleh melawati pautan ini https://www.tonton.com.my/tontonup untuk menaik taraf akaun anda. Sebaik sahaja anda mengklik pautan, anda akan melihat halaman yang menunjukkan ruangan "SUBSCRIBE NOW". Sila klik ke ruangan "SUBSCRIBE NOW" di bahagian ATAS sekali untuk meneruskan proses langganan dan pembayaran. Sila pilih kaedah pembayaran pilihan anda dan ikuti arahan seterusnya di dalam Langkah 2. Sebaik sahaja anda telah membuat pembayaran langganan, kami mencadangkan anda untuk klik butang Pembaharuan

Chunk 2:
Kami menyarankan anda supaya mengikuti langkah-langkah berikut untuk mengaktifkan langganan anda selepas bayaran dilakukan: 1. Klik pada ikon 'setting' pada bahagian atas sebelah kiri (telefon bimbit) aplikasi Tonton atau sebelah kanan (komputer) laman web (website) anda. 2. Pilih Tetapan Aplikasi (App Settings) daripada menu. 3. Klik butang Akaun (Account) 4. Klik butang P

In [19]:
response = ask_chatbot("Saya telah melanggan tetapi kenapa masih mempunyai iklan?")
print(response)

🧾 Retrieved content:

Chunk 1:
Untuk makluman anda, langganan TontonUp memberikan anda akses untuk menonton filem dan drama eksklusif dan saluran TV premium yang hanya boleh didapati di aplikasi Tonton. Untuk menonton tanpa iklan, anda perlulah memilih salah satu daripada saluran TV premium dan bahan tontonan eksklusif yang bertanda logo TontonUp. Untuk cerita tontonan yang bertanda TontonUp, iklan hanya disiarkan pada episod pertama sehingga ke tiga bagi drama yang ditonton sahaja dan tidak akan mempunyai iklan pada episod seterus .

Iklan hanya dipaparkan pada episod pertama hingga ketiga bagi drama yang bertanda TontonUp.  Episod seterusnya bebas daripada iklan.  Sekiranya iklan masih dipaparkan selepas episod ketiga, pastikan anda menonton kandungan yang mempunyai logo TontonUp untuk tontonan tanpa iklan.



In [25]:
response = ask_chatbot("Bagaimana saya nak membatalkan langganan bulanan TontonUp saya?")
print(response)

🧾 Retrieved content:

Chunk 1:
Untuk makluman, anda boleh membatalkan pembayaran berulang automatik anda pada Profil Tonton anda. Bagi rujukan mudah anda, berikut adalah langkah-langkah untuk membatalkan langganan TontonUp anda: Anda boleh menggunakan TELEFON BIMBIT atau DESKTOP melalui laman web (komputer). Telefon Bimbit 1) Pergi ke Tetapan (Settings) > klik pada Akaun (Account) > Klik pada Pembaharuan Kelayakan (Refresh Entitlements) > Kemudian klik pada Urus Akaun (Manage account) 2) Klik pada pelan langganan (Subscription Plan) 3) Klik pada batalkan langganan berulang (Cancel Recurring Subscription) Laman Web (Website) 1) Log masuk ke Akaun (Account) > Klik pada Tetapan (Settings) > Pilih Akaun (Account) 2) Pergi ke pelan langganan (Subscription Plan) 3) Klik pada batalkan langganan berulang (Cancel Recurring Subscription) Jika anda menghadapi masalah untuk membatalkan langganan anda, sila sertakan resit pembayaran terkini dan butiran berikut untuk kami menyemak dengan lebih lanju

In [23]:
response = ask_chatbot("Bagaimana saya nak menukar kata laluan?")
print(response)

🧾 Retrieved content:

Chunk 1:
Kami mencadangkan anda untuk membuat tetapan semula kata laluan, kemudian log masuk semula. Anda boleh ke pautan di bawah bagi menetapkan semula kata laluan bagi akaun anda - https://oauth.revmedia.my/forgot?client_id=551835651455588&redirect_uri=https%3A%2F%2Fheadend-api.tonton.com.my%2Fv100%2Fapi%2Fauth.class.api.php%2Fcallback&response_type=code&code_challenge=DJfXvpi3ZqdivuoclgDAwSuLyMfyNwkz4cbtfzt_vU8&code_challenge_method=S256&state=TONTON_web.B3Wa0gmr3pAG8zKMqutaT2bXabyU01RB

Anda boleh menetapkan semula kata laluan akaun anda dengan melayari pautan ini:  `https://oauth.revmedia.my/forgot?client_id=551835651455588&redirect_uri=https%3A%2F%2Fheadend-api.tonton.com.my%2Fv100%2Fapi%2Fauth.class.api.php%2Fcallback&response_type=code&code_challenge=DJfXvpi3ZqdivuoclgDAwSuLyMfyNwkz4cbtfzt_vU8&code_challenge_method=S256&state=TONTON_web.B3Wa0gmr3pAG8zKMqutaT2bXabyU01RB`
Selepas itu, log masuk semula.



In [31]:
response = ask_chatbot("Boleh saya melanggan apabila saya di luar negara Malaysia?")
print(response)

🧾 Retrieved content:

Chunk 1:
Untuk makluman, platform kami hanya boleh di tonton di Malaysia sahaja pada ketika ini.

Maklumat tidak tersedia.



In [29]:
response = ask_chatbot("Mengapa ralat muncul di skrin semasa saya menikmati Tonton? ")
print(response)

🧾 Retrieved content:

Chunk 1:
Jika anda ingin mengaktifkan semula langganan untuk kandungan eksklusif, anda boleh melawati pautan ini https://www.tonton.com.my/tontonup untuk menaik taraf akaun anda. Sebaik sahaja anda mengklik pautan, anda akan melihat halaman yang menunjukkan ruangan "SUBSCRIBE NOW". Sila klik ke ruangan "SUBSCRIBE NOW" di bahagian ATAS sekali untuk meneruskan proses langganan dan pembayaran. Sila pilih kaedah pembayaran pilihan anda dan ikuti arahan seterusnya di dalam Langkah 2. Sebaik sahaja anda telah membuat pembayaran langganan, kami mencadangkan anda untuk klik butang Pembaharuan

Chunk 2:
Untuk makluman anda, langganan TontonUp memberikan anda akses untuk menonton filem dan drama eksklusif dan saluran TV premium yang hanya boleh didapati di aplikasi Tonton. Untuk menonton tanpa iklan, anda perlulah memilih salah satu daripada saluran TV premium dan bahan tontonan eksklusif yang bertanda logo TontonUp. Untuk cerita tontonan yang bertanda TontonUp, iklan hanya

In [33]:
import pickle, faiss

# Save FAQ data
with open("faq_data.pkl", "wb") as f:
    pickle.dump(faq_data, f)

# Save FAISS index
faiss.write_index(index, "faq_index.faiss")
print("Saved faq_data.pkl and faq_index.faiss")

Saved faq_data.pkl and faq_index.faiss
